<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=029396124ce443022175da2e3f7a24ddb2ef349a9aa5ef0f50febe37b7d34862
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 14:42:14
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.43 C
-------------------
Today PnL: 15.83 K (0.11%)
Current PnL: -22.03 L (-14.44%)
CY Booked + Current PnL: -8.84 L (-5.79%)
-------------------
Total profit:  1.65 L
Total loss:  -23.68 L
-------------------
Total Booked + Current PnL: 18.14 L (14.44%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.22%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 89.49 L (62.54%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.28,51.0,M-SC,2.35,86722.0,-14055.0,14144.0,0.20,-13.95,16.31,0.09,245.0,-0.99,0.61,13.93,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198852.0,-1286.0,22172.0,0.27,-0.64,11.15,10.44,4.0,-0.06,1.40,4.89,X40,NTT,FMCG
34,ICICIGI,2252.93,-14.07,54.0,X-MC,6.59,185628.0,11308.0,23890.0,-1.45,6.49,12.87,20.19,91.0,0.47,1.31,22.58,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-3.64,60.0,X-LC,8.22,296338.0,30912.0,25870.0,0.28,11.65,8.73,21.40,11.0,1.19,2.09,19.06,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3667.57,56.0,M-SC,3.85,169201.0,-27931.0,117764.0,11.64,-14.17,69.60,45.57,236.0,-0.24,1.19,19.96,XY24,NTT,AUTO
39,INDUSINDBK,1800.00,282.35,70.0,L-MC,7.38,46894.0,-30312.0,55705.0,2.92,-39.26,118.79,32.89,258.0,-0.54,0.33,29.14,XR,NTT,BANKS
13,BERGEPAINT,680.00,-17.54,56.0,X-MC,1.10,221758.0,-5581.0,53643.0,2.23,-2.45,24.19,21.14,106.0,-0.10,1.56,24.99,XY24,NTT,PAINTS
38,INDIGOPNTS,1408.00,123.21,76.0,M-SC,5.33,146444.0,-28115.0,28147.0,2.22,-16.11,19.22,0.02,221.0,-1.00,1.03,27.38,OX40N,NTT,PAINTS
14,BSOFT,836.99,-13.25,60.0,H-SC,9.73,97448.0,-43205.0,115983.0,2.06,-30.72,119.02,51.74,131.0,-0.37,0.69,12.75,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-44.58,38.0,M-SC,8.40,114834.0,36.0,94003.0,-2.88,0.03,81.86,81.92,223.0,0.00,0.81,35.95,XR,NTT,DURABLES
83,VOLTAS,1530.00,-6.81,36.0,X-MC,4.02,195645.0,3903.0,33847.0,-2.71,2.04,17.30,19.69,99.0,0.12,1.38,8.97,XY25,NTT,AC
80,VAIBHAVGBL,521.00,66.39,55.0,H-SC,5.58,144102.0,-38673.0,149736.0,-2.56,-21.16,103.91,60.77,125.0,-0.26,1.01,29.77,XR,NTT,JEWELLERY
26,GREENPANEL,537.00,197.80,37.0,M-SC,5.24,132858.0,-50220.0,129204.0,-2.45,-27.43,97.25,43.14,230.0,-0.39,0.93,24.97,XY24,NTT,MISC
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248250.0,-5646.0,87955.0,-2.37,-2.22,35.43,32.42,105.0,-0.06,1.75,12.54,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.58,38.0,M-SC,8.40,114834.0,36.0,94003.0,-2.88,0.03,81.86,81.92,223.0,0.00,0.81,35.95,XR,NTT,DURABLES
85,WIPRO,420.0,-14.70,49.0,M-LC,5.67,149823.0,-1122.0,110584.0,0.75,-0.74,73.81,72.51,53.0,-0.01,1.05,5.23,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.28,51.0,M-SC,2.35,86722.0,-14055.0,14144.0,0.20,-13.95,16.31,0.09,245.0,-0.99,0.61,13.93,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,123.21,76.0,M-SC,5.33,146444.0,-28115.0,28147.0,2.22,-16.11,19.22,0.02,221.0,-1.00,1.03,27.38,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,40.0,H-SC,2.27,219465.0,-50202.0,86535.0,-0.80,-18.62,39.43,13.47,138.0,-0.58,1.54,11.59,XY24,NTT,PAINTS
67,SIS,528.0,2027.98,48.0,H-SC,2.19,85884.0,-25148.0,48756.0,0.61,-22.65,56.77,21.26,156.0,-0.52,0.60,16.04,OX40N,NTT,MISC
43,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97944.0,-29861.0,29844.0,0.04,-23.36,30.47,-0.01,232.0,-1.00,0.69,10.91,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.18,70.0,M-MC,10.50,248691.0,23729.0,143917.0,0.06,10.55,57.87,74.53,192.0,0.16,1.75,46.20,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,55.0,M-LC,9.20,211758.0,6682.0,105879.0,0.45,3.26,50.00,54.89,52.0,0.06,1.49,28.14,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.58,38.0,M-SC,8.40,114834.0,36.0,94003.0,-2.88,0.03,81.86,81.92,223.0,0.00,0.81,35.95,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.71,50.0,H-SC,5.07,129214.0,5878.0,125751.0,-0.56,4.77,97.32,106.72,119.0,0.05,0.91,29.26,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,54.0,M-SC,35.17,159309.0,10089.0,134090.0,-1.24,6.76,84.17,96.62,208.0,0.08,1.12,72.41,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.80,55.0,M-LC,9.20,211758.0,6682.0,105879.0,0.45,3.26,50.00,54.89,52.0,0.06,1.49,28.14,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.71,50.0,H-SC,5.07,129214.0,5878.0,125751.0,-0.56,4.77,97.32,106.72,119.0,0.05,0.91,29.26,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.58,38.0,M-SC,8.40,114834.0,36.0,94003.0,-2.88,0.03,81.86,81.92,223.0,0.00,0.81,35.95,XR,NTT,DURABLES
85,WIPRO,420.00,-14.70,49.0,M-LC,5.67,149823.0,-1122.0,110584.0,0.75,-0.74,73.81,72.51,53.0,-0.01,1.05,5.23,XR,NTT,IT
47,KPIGREEN,731.05,3.65,46.0,H-SC,9.83,123719.0,-1578.0,60511.0,1.37,-1.26,48.91,47.03,141.0,-0.03,0.87,54.91,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248250.0,-5646.0,87955.0,-2.37,-2.22,35.43,32.42,105.0,-0.06,1.75,12.54,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-32.66,31.0,X-MC,0.69,194850.0,-5970.0,73770.0,-2.21,-2.97,37.86,33.76,80.0,-0.08,1.37,2.07,X40,ATH,FMCG
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47970.0,-17036.0,167967.0,-2.17,-26.21,350.15,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-32.66,31.0,X-MC,0.69,194850.0,-5970.0,73770.0,-2.21,-2.97,37.86,33.76,80.0,-0.08,1.37,2.07,X40,ATH,FMCG
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198852.0,-1286.0,22172.0,0.27,-0.64,11.15,10.44,4.0,-0.06,1.40,4.89,X40,NTT,FMCG
66,SIEMENS,4671.50,-6.25,36.0,H-LC,1.63,150675.0,-35420.0,82901.0,-1.34,-19.03,55.02,25.51,15.0,-0.43,1.06,11.04,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.30,36.0,X-MC,3.32,302862.0,-25129.0,131654.0,-1.06,-7.66,43.47,32.48,92.0,-0.19,2.13,2.89,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.81,60.0,X-LC,3.61,244223.0,10079.0,38538.0,0.74,4.30,15.78,20.77,86.0,0.26,1.72,11.52,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47970.0,-17036.0,167967.0,-2.17,-26.21,350.15,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
1,ABB,7934.00,-42.98,32.0,H-LC,5.00,238200.0,-23419.0,142634.0,-0.59,-8.95,59.88,45.57,7.0,-0.16,1.68,1.36,AR,NTT,ELECTRICAL
52,PAGEIND,51769.93,-29.12,38.0,X-MC,7.64,160220.0,-3520.0,46864.0,-0.36,-2.15,29.25,26.47,82.0,-0.08,1.13,1.51,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-16.17,36.0,X-MC,6.36,176464.0,-24042.0,121195.0,-0.61,-11.99,68.68,48.45,90.0,-0.20,1.24,1.76,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47970.0,-17036.0,167967.0,-2.17,-26.21,350.15,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.85,40.0,X-SC,4.04,80652.0,-64508.0,140439.0,0.25,-44.44,174.13,52.31,136.0,-0.46,0.57,7.96,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.48,36.0,X-SC,0.93,106500.0,-21438.0,68575.0,-1.43,-16.76,64.39,36.84,143.0,-0.31,0.75,9.03,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-28.36,53.0,X-LC,9.35,285986.0,-59970.0,126663.0,0.57,-17.33,44.29,19.28,1.0,-0.47,2.01,5.72,X40,ATH,IT
40,INFY,2275.00,-16.37,63.0,X-LC,4.70,325677.0,12789.0,158898.0,1.02,4.09,48.79,54.87,3.0,0.08,2.29,12.71,X40,BTT,IT
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198852.0,-1286.0,22172.0,0.27,-0.64,11.15,10.44,4.0,-0.06,1.40,4.89,X40,NTT,FMCG
82,VBL,671.64,-16.58,53.0,X-LC,5.73,300211.0,-15631.0,128310.0,1.58,-4.95,42.74,35.67,5.0,-0.12,2.11,8.22,X40N,ATH,FMCG
1,ABB,7934.00,-42.98,32.0,H-LC,5.00,238200.0,-23419.0,142634.0,-0.59,-8.95,59.88,45.57,7.0,-0.16,1.68,1.36,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,27.22,76416.0,-37133.0,77188.0,-2.07,-32.70,101.01,35.27,268.0,-0.48,0.54,87.03,XR,NTT,HOTELS
7,ASIANTILES,137.00,7588.89,69.0,L-SC,14.08,84343.0,-9467.0,86080.0,0.31,-10.09,102.06,81.67,269.0,-0.11,0.59,63.33,XR,NTT,CERAMICS
50,MASFIN,398.61,-20.51,45.0,H-SC,7.90,90330.0,-7650.0,29249.0,-1.28,-7.81,32.38,22.05,152.0,-0.26,0.64,31.30,XR,ATH,FINANCE
47,KPIGREEN,731.05,3.65,46.0,H-SC,9.83,123719.0,-1578.0,60511.0,1.37,-1.26,48.91,47.03,141.0,-0.03,0.87,54.91,MH,ATH,POWER
80,VAIBHAVGBL,521.00,66.39,55.0,H-SC,5.58,144102.0,-38673.0,149736.0,-2.56,-21.16,103.91,60.77,125.0,-0.26,1.01,29.77,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7588.89,69.0,L-SC,14.08,84343.0,-9467.0,86080.0,0.31,-10.09,102.06,81.67,269.0,-0.11,0.59,63.33,XR,NTT,CERAMICS
80,VAIBHAVGBL,521.00,66.39,55.0,H-SC,5.58,144102.0,-38673.0,149736.0,-2.56,-21.16,103.91,60.77,125.0,-0.26,1.01,29.77,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,-7.44,74.0,X-LC,6.18,233552.0,-18216.0,71420.0,0.14,-7.24,30.58,21.13,27.0,-0.26,1.64,25.63,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,123.21,76.0,M-SC,5.33,146444.0,-28115.0,28147.0,2.22,-16.11,19.22,0.02,221.0,-1.00,1.03,27.38,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,22.45,68.0,X-SC,6.67,216431.0,25425.0,32270.0,1.27,13.31,14.91,30.21,157.0,0.79,1.52,36.97,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.78
2,50,76.22


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    30.48
MC    25.99
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.86
X40      19.29
X40N     12.46
XY25     10.21
XR        9.69
AR        7.72
OX40N     6.59
X200      1.80
X5K       1.49
SR        1.02
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    21.57
X-LC    21.01
M-SC    12.61
X-SC     5.57
H-LC     4.54
M-LC     3.94
H-MC     2.34
M-MC     1.75
L-SC     1.46
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.99,-4.21,37.91
IT,12.65,-17.03,79.67
FINANCE,12.11,-9.87,59.67
MISC,7.68,-18.93,78.93
PAINTS,5.77,-12.43,29.19
ELECTRICAL,5.62,-13.20,53.34
INSURANCE,4.47,0.27,34.87
AUTO,2.84,-42.33,101.74
BANKS,2.71,-38.76,129.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3287412.0,21
XR,1279314.0,13
AR,1191356.0,9
X40,951429.0,13
X40N,697441.0,9
OX40N,547626.0,9
XY25,483466.0,7
SR,276395.0,2
X5K,105879.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3437348.0,24
X-MC,1344569.0,15
M-SC,1251895.0,15
X-LC,1053111.0,12
X-SC,557871.0,6
H-LC,294153.0,3
M-LC,263075.0,3
H-MC,256270.0,2
L-SC,253405.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193379.0      6
           AR         841981.0      5
           XR         806479.0      7
M-SC       XY24       725488.0      6
X-MC       XY24       565620.0      4
X-LC       X40        435380.0      5
X-MC       X40        429056.0      7
X-LC       XY24       308458.0      2
X-SC       X40N       302911.0      4
H-SC       SR         276395.0      2
           OX40N      258603.0      3
X-LC       X40N       233413.0      3
H-LC       AR         225535.0      2
X-MC       XY25       188776.0      2
H-MC       XY24       182583.0      1
X-SC       XY24       167967.0      1
L-SC       XR         163268.0      2
X-MC       X40N       161117.0      2
M-MC       XY24       143917.0      1
M-SC       XR         143278.0      2
           XY25       134090.0      1
           OX40N      125199.0      4
           AR         123840.0      2
M-LC       XR         110584.0      1
           X5K        105879.0      1
L-SC       OX40N       90137.0      1
X-SC       X40         86993.0      1
X-LC       XY25        75860.0      2
H-MC       OX40N       73687.0      1
H-LC       X200        68618.0      1
H-SC       MH          60511.0      1
L-MC       XR          55705.0      1
M-LC       XY25        46612.0      1
L-LC       XY25        38128.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
